# REMOVING OUTLIERS AND FEATURE ENGINEERING DISTANCE TO WATER
## last changes before master data



In [1]:
#importing the libraries used in our analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
import geopy.distance as geo
%matplotlib inline

# Outlier removal
begin reading the file that has already had duplicates (resales) extracted from it AND encoded categorical location variables: Suburban, Urban and Rural created by binning zip codes

In [5]:
#reading the dupes-free and categorical var encoded data (see link above)
df = pd.read_csv('data/ohe_zip_data.csv')
df.drop('Unnamed: 0', axis = 1, inplace=True)

In [7]:
#inspecting the data
df.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,label_rural,label_suburban,label_urban
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,1955,0.0,98178,47.5112,-122.257,1340,5650,0,1,0
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,1951,1991.0,98125,47.7210,-122.319,1690,7639,0,0,1


In [8]:
#checking the nature of our data columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21420 entries, 0 to 21419
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              21420 non-null  int64  
 1   date            21420 non-null  object 
 2   price           21420 non-null  float64
 3   bedrooms        21420 non-null  int64  
 4   bathrooms       21420 non-null  float64
 5   sqft_living     21420 non-null  int64  
 6   sqft_lot        21420 non-null  int64  
 7   floors          21420 non-null  float64
 8   waterfront      19067 non-null  object 
 9   view            21357 non-null  object 
 10  condition       21420 non-null  object 
 11  grade           21420 non-null  int64  
 12  sqft_above      21420 non-null  int64  
 13  sqft_basement   21420 non-null  object 
 14  yr_built        21420 non-null  int64  
 15  yr_renovated    17616 non-null  float64
 16  zipcode         21420 non-null  int64  
 17  lat             21420 non-null 

In [11]:
#Choosing the columns to remove outliers on.
outlier_columns = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_living15', 'sqft_lot15']
for col in outlier_columns:
    df_filtered_std = df[((df[col] - df[col].mean()) / df[col].std()).abs() < 3]
df_filtered_std.shape

(21059, 24)

In [12]:
#writing down this cleaned data for safe-keeping
df_filtered_std.to_csv('data/df_no_oultliers.csv')

## Creating a coordinates column from latitude and longitude

In [13]:
df_water = df_filtered_std.copy() # making a copy to add features to
df_water['coordinates'] = pd.Series([(x,y) for x, y in df_water[['lat', 'long']].values])

In [15]:
#verifying the creation of coordinates column
df_water.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,label_rural,label_suburban,label_urban,coordinates
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,0.0,98178,47.5112,-122.257,1340,5650,0,1,0,"(47.5112, -122.257)"
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,1991.0,98125,47.7210,-122.319,1690,7639,0,0,1,"(47.721, -122.319)"


# Creating a function that maps distance to waterfront view

In [22]:
#First create the list of waterfront coordinates
water_view_df = df[df['waterfront'] =='YES']


In [17]:
#defining the function that gives the minimum distance to a waterfront property 
def closest_water(coor_1):
    dist_list = []
    for row in df:
        for x, y  in water_view_df[['lat', 'long']].values:
            coor_2 = (x,y)
            distance = geo.distance(coor_1, coor_2)
            dist_list.append(distance)
        return min(dist_list)

### mapping the function to the dataframe

df_water.loc[:, 'closest_water'] = df_water.loc[:,'coordinates'].map(closest_water)


mapping takes almost 10 minutes so we immediately save this down
### saving our master data set

In [25]:
df_water.to_csv('data/master_data.csv')